In [1]:
%matplotlib inline

import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import os
from natsort import natsorted
import collections
import numpy as np
import community as cm
from scipy.optimize import curve_fit
from functools import reduce
from math import floor
import pickle

In [12]:
def fractal_model(generation,m,x,e):
	"""
	Returns the fractal model introduced by
	Song, Havlin, Makse in Nature Physics 2, 275.
	generation = number of generations
	m = number of offspring per node
	x = number of connections between offsprings
	e = probability that hubs stay connected
	1-e = probability that x offsprings connect.
	If e=1 we are in MODE 1 (pure small-world).
	If e=0 we are in MODE 2 (pure fractal).
	"""
	G=nx.Graph()
	G.add_edge(0,1) #This is the seed for the network (generation 0)
	node_index = 2
	for n in range(1,generation+1):
		all_links = list(G.edges())
		while all_links:
			link = all_links.pop()
			new_nodes_a = range(node_index,node_index + m)
			#random.shuffle(new_nodes_a)
			node_index += m
			new_nodes_b = range(node_index,node_index + m)
			#random.shuffle(new_nodes_b)
			node_index += m
			G.add_edges_from([(link[0],node) for node in new_nodes_a])
			G.add_edges_from([(link[1],node) for node in new_nodes_b])
			repulsive_links = list(zip(new_nodes_a,new_nodes_b))
			G.add_edges_from([repulsive_links.pop() for i in range(x-1)])
			if np.random.random() > e:
				G.remove_edge(link[0], link[1])
				new_edge = repulsive_links.pop()
				G.add_edge(new_edge[0], new_edge[1])
	return G

In [8]:
refs = []
for f in tqdm_notebook(natsorted(os.listdir('time_graphs2/'))):
    refs.append(nx.read_graphml('time_graphs2/' + f + '/2495.graphml'))
    
len(refs)

10

In [3]:
true_lens = []
true_clusts = []

for r in tqdm_notebook(refs):
    true_lens.append(nx.average_shortest_path_length(r))
    true_clusts.append(nx.average_clustering(r))

print(true_lens[:5])
print(true_clusts[:5])
    
true_len = sum(true_lens) / len(true_lens)
true_clus = sum(true_clusts) / len(true_clusts)
    
lens = []
clusts = []

for i in tqdm_notebook(range(len(refs))):
    ref = refs[i]
    temp_lens = []
    temp_clusts = []
    for _ in range(10):
        rand = nx.erdos_renyi_graph(ref.number_of_nodes(), ref.number_of_edges() / ref.number_of_nodes()**2)
    #     rand = nx.random_reference(ref)
        while not nx.is_connected(rand):
            rand = nx.erdos_renyi_graph(ref.number_of_nodes(), ref.number_of_edges() / ref.number_of_nodes()**2)
    #         rand = nx.random_reference(ref)
        cur_len = nx.average_shortest_path_length(rand)
        cur_clust = nx.average_clustering(rand)
        lens.append(cur_len)
        clusts.append(cur_clust)
        temp_lens.append(cur_len)
        temp_clusts.append(cur_clust)
        
    ref_len = sum(temp_lens) / len(temp_lens)
    ref_clus = sum(temp_clusts) / len(temp_clusts)
    true_len = true_lens[i]
    true_clus = true_clusts[i]
    print(true_len / ref_len, true_clus / ref_clus)
        
    
print("==========")
ref_len = sum(lens) / len(lens)
ref_clus = sum(clusts) / len(clusts)

print(true_len, true_clus)
print(ref_len, ref_clus)
print(true_len / ref_len, true_clus / ref_clus)


[4.148023852936254, 4.219277785329165, 4.327945004675613, 4.137659305379901, 4.113465740232575]
[0.2817407728680952, 0.2973902143934266, 0.28411878999151047, 0.29706132037749766, 0.28604746200963604]


1.0105242621012662 46.38396582855988
1.0067275978015806 56.91291811738465
1.0251854530346285 55.10634945090349
0.9978647880775205 52.149798133625566
0.9893348377134459 51.66143278993813
1.0318836404663008 45.398068382088546
0.9962834505344778 49.64074362444728
1.0094267562256451 50.55162265389754
1.0114051694407031 51.954324336930895
1.0313463432138532 58.427923471703075
1.014858653909215 57.33669314396059
1.0119807510928276 50.44443367294479
1.0293909467955713 50.005949390398484
1.0195312138617258 53.418959582240745
0.9917311834239348 50.895008929409876
1.0008804170712735 55.95092245039407
0.9775728388061068 49.74508474030382
0.9885651850725579 53.1997103120914
0.9949537744412298 60.446007285417494
0.9939677757757632 51.50230582724532
0.9938508080917823 47.30598023179903
0.988058908773853 46.06617799942547
0.9515107586825622 44.05718375566458
1.0131379260762516 58.75002425389177
1.002526090270371 53.76498807240356
1.0021230478222996 50.69627779299733
1.0096683053920779 60.182749261868

In [214]:
def scale(lb, a):
    return  a * lb**-2.8

r2s = []
# G = nx.read_edgelist('real_graphs/facebook_combined.txt')
# G = nx.read_graphml('0.graphml')
# G = nx.read_graphml('time_graphs/' + str(0) + '/4990.graphml')
Gs = [nx.read_graphml('time_graphs/' + str(i) + '/4990.graphml') for i in range(5)]
for idx, G in enumerate(Gs):
    degree_sequence = [d for (n, d) in G.degree()]
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    tot = sum(cnt)
    per = [c / tot for c in cnt]

    vals = sorted([(deg[i], per[i]) for i in range(len(deg))])
    x = np.array([v[0] for v in vals])[15:]
    y = np.array([v[1] for v in vals])[15:]

    popt = curve_fit(scale, x, y)
    print(popt)
    popt = popt[0]

    plt.xscale('log')
    plt.yscale('log')
    plt.plot(deg, per, 'x')
    xx = np.linspace(10, max(x), 100)
    yy = [scale(x, *popt) for x in xx]
    plt.plot(xx, yy, lw=5)

    plt.savefig('deg_plots/' + str(idx) + '.png')
    plt.cla()
    plt.clf()
    # plt.show()
    
    y_fit = scale(x, *popt)

    ss_res = np.sum((y - y_fit) ** 2)

    # total sum of squares
    ss_tot = np.sum((y - np.mean(y)) ** 2)

    # r-squared
    r2 = 1 - (ss_res / ss_tot)
    r2s.append(r2)
    print(r2)


print(np.average(r2s))

(array([87.44230476]), array([[5.70267201]]))
0.9310067117697917
(array([85.08632065]), array([[3.11247002]]))
0.9578436889071638
(array([87.05117235]), array([[3.8232854]]))
0.953004553766293
(array([91.45365965]), array([[6.10660003]]))
0.9329432649013116
(array([87.20336433]), array([[3.91332244]]))
0.9508852478810319
0.9451366934451183


<Figure size 432x288 with 0 Axes>

In [218]:
def scale(lb, db, a):
    return  a * lb**-db

G = nx.read_edgelist('real_graphs/facebook_combined.txt')
# G = nx.read_graphml('0.graphml')
# G = nx.read_graphml('time_graphs/' + str(0) + '/4990.graphml')
degree_sequence = [d for (n, d) in G.degree()]
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

tot = sum(cnt)
per = [c / tot for c in cnt]

vals = sorted([(deg[i], per[i]) for i in range(len(deg))])
x = np.array([v[0] for v in vals])[15:]
y = np.array([v[1] for v in vals])[15:]

popt = curve_fit(scale, x, y)[0]
print(popt)

plt.xscale('log')
plt.yscale('log')
plt.plot(deg, per, 'x')
xx = np.linspace(10, max(x) * 0.6, 100)
yy = [scale(x, *popt) for x in xx]
plt.plot(xx, yy, lw=5)

plt.savefig('deg_plots/' + 'facebook' + '.png')
plt.cla()
plt.clf()
# plt.show()

[1.2530342  0.70328266]


<Figure size 432x288 with 0 Axes>

In [221]:
def scale(lb, db, a):
    return  a * lb**-db

# G = nx.read_edgelist('real_graphs/facebook_combined.txt')
G = nx.barabasi_albert_graph(2500, 5)
# G = nx.read_graphml('0.graphml')

print('generated model')
degree_sequence = [d for (n, d) in G.degree()]
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

tot = sum(cnt)
per = [c / tot for c in cnt]

vals = sorted([(deg[i], per[i]) for i in range(len(deg))])
x = np.array([v[0] for v in vals])[15:]
y = np.array([v[1] for v in vals])[15:]

popt = curve_fit(scale, x, y)[0]
print(popt)

plt.xscale('log')
plt.yscale('log')
plt.plot(deg, per, 'x')
xx = np.linspace(min(deg), 10**1.7, 100)
yy = [scale(x, *popt) for x in xx]
plt.plot(xx, yy, lw=5)

plt.savefig('deg_plots/' + 'ba' + '.png')
plt.cla()
plt.clf()

generated model
[ 2.73962012 25.23877724]


<Figure size 432x288 with 0 Axes>

In [213]:
def lesk(x, a=1.25):
    t1 = 4 * (x**(a-1)) - 1
    t2 = 2 * (x**(a-1)) - 1
    
    return t1/t2

print(len(Gs[0]))
print(np.average(list(range(len(Gs[0])))))
lesk(3569)

3569
1784.0


2.069163590049465

In [3]:
mods = []
nodes = []
edges = []
Gs = [nx.read_graphml('time_graphs/' + str(i) + '/4990.graphml') for i in range(5)]
# Gs = [nx.read_graphml('0.graphml')]
for idx, G in tqdm_notebook(enumerate(Gs)):
    for _ in range(5):
        coms = cm.best_partition(G)
        mod = cm.modularity(coms, G)
        mods.append(mod)
    nodes.append(G.number_of_nodes())
    edges.append(G.number_of_edges())
    
print(np.average(mods), np.average(nodes), np.average(edges))


0.7412424033092034 3525.2 27409.8


In [4]:
mods = []
nodes = []
edges = []
for _ in tqdm_notebook(range(5)):
    r = nx.connected_watts_strogatz_graph(3500, 17, 0.15)
    for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    edges.append(r.number_of_edges())
    nodes.append(r.number_of_nodes())
    
print(np.average(mods), np.average(nodes), np.average(edges))


0.7920036678571428 3500.0 28000.0


In [5]:
mods = []
nodes = []
edges = []
for _ in tqdm_notebook(range(5)):
    r = nx.erdos_renyi_graph(3500, 0.004)
    for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    edges.append(r.number_of_edges())
    nodes.append(r.number_of_nodes())
    
print(np.average(mods), np.average(nodes), np.average(edges))


0.22395862606543576 3500.0 24503.0


In [13]:
mods = []
nodes = []
edges = []
for _ in tqdm_notebook(range(5)):
    r = fractal_model(4,3,2,0)
    for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    edges.append(r.number_of_edges())
    nodes.append(r.number_of_nodes())
    
print(np.average(mods), np.average(nodes), np.average(edges))


0.9259720611572265 3512.0 4096.0


In [7]:
r = nx.read_graphml('cora.graphml')
r = nx.Graph(r)
mods = []

for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    
print(np.average(mods), r.number_of_nodes(), r.number_of_edges())

0.8128593979226375 2708 5278


In [8]:
r = nx.read_graphml('0.graphml')
mods = []

for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    
print(np.average(mods), r.number_of_nodes(), r.number_of_edges())

0.6527208387367345 3015 5539


In [9]:
r = nx.read_edgelist('real_graphs/email.txt', data=False)
mods = []

for _ in range(5):
        coms = cm.best_partition(r)
        mod = cm.modularity(coms, r)
        mods.append(mod)
    
print(np.average(mods), r.number_of_nodes(), r.number_of_edges())

0.569809563917796 1133 5452
